In [6]:
import re
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
#Загрузка модели
model_name = 'sberbank-ai/sbert_large_nlu_ru'
model = SentenceTransformer(model_name)

In [31]:
def semantic_search(document,query, model,threshhold = 0.65, context_window_chars = 10, context_weight_alpha = 0.8):
    # Выделение кандидатов
    words_with_indices = []
    for match in re.finditer(r'\w+',document):
        words_with_indices.append({
            'text':match.group(0),
            'start':match.start(),
            'end': match.end()
        })
    candidates = []
    for i in range(len(words_with_indices)):
        candidates.append(words_with_indices[i])    

    for i in range(len(words_with_indices)-1):
        word1 = words_with_indices[i]
        word2 = words_with_indices[i+1]
        candidates.append({
        'text': f'{word1['text']} {word2['text']}',
        'start': word1['start'],
        'end': word2['end']
        })

    for i in range(len(words_with_indices)-2):
        word1 = words_with_indices[i]
        word2 = words_with_indices[i+1]
        word3 = words_with_indices[i+2]
        candidates.append({
        'text': f'{word1['text']} {word2['text']} {word3['text']}',
        'start': word1['start'],
        'end': word3['end']
        })
    #Получение эмбеддингов
    query_embedding = model.encode(query)
    candidate_texts = [cand['text'] for cand in candidates]
    candidate_embeddings = model.encode(candidate_texts)
    #Вычисление сходства
    similarities = cosine_similarity([query_embedding], candidate_embeddings)[0]
    best_candidate_index = np.argmax(similarities)
    original_similarity = similarities[best_candidate_index]
    best_candidate = candidates[best_candidate_index]

    context_start = max(0, best_candidate['start'] - context_window_chars)
    context_end = min(len(document), best_candidate['end'] + context_window_chars)
    context_window_text = document[context_start:context_end]

    context_embedding = model.encode(context_window_text)

    context_similarity = cosine_similarity([query_embedding], [context_embedding])[0][0]
    context_similarity = float(context_similarity)

    final_score = context_weight_alpha * original_similarity + (1-context_weight_alpha)*context_similarity
    final_score = float(final_score)
    original_similarity = float(original_similarity)
    
    if final_score >= threshhold:
        position_str = f'{best_candidate['start']} - {best_candidate['end']}'
        probability = final_score
        return best_candidate, position_str, probability
    else:
        print(f"Макс. сходство {final_score:.4f} для '{best_candidate['text']}' не достигло порога {threshhold}")
        return None 

In [33]:
# --- Тестирование ---
examples = [
    ("доченька твоя совсем большая стала", "дочь"),
    ("вся дорога забита деревьями и цветами", "дерево"),
    ("в следующее воскресенье я собираюсь в питер", "санкт петербург"),
    ("у меня сломалась стиралка прикинь", "стиральная машина"),
    ("садись в машину и поехали уже", "автомобиль"),
    ("сколько стоит ремонт стиральной машины", "автомобиль"), # Ожидается НЕ найти?
    ("ты когда собираешься звонить преподу", "учитель"),
    ("возьмешь пистолет в тумбочке понял меня", "оружие"),
    ("ты не мог бы набрать меня после обеда", "звонить"),
    ("ты возьми корзину прежде чем набрать продукты", "звонить"), # Ожидается НЕ найти?
    ("его сегодня утром отвезли в ближайший госпиталь", "больница"),
    ("в этому году смартфоны подорожают на 30 процентов", "телефон"),
    ("я еще долгу не смогу вернуться в рф", "россия"),
    ("мужчина средних лет с серым рюкзаком и шапкой", "портфель"),
    ("в какой университет собирается поступать твой сын", "институт"),
    ("он вообще не собирается переезжать в другое государство", "страна"),
    ("сегодня ночью наше судно отплывает по расписанию", "корабль"),
    ("его оригинальный портрет выставлен на продажу", "подлинная картина"),
]
for document, query in examples:
    result = semantic_search(document, query, model, threshhold = 0.6)
    print(f"Документ: '{document}'")
    print(f"Запрос: '{query}'")
    if result:
        best_candidate, position_str, probability = result
        print(f'Результат: Слово - {best_candidate}, Позиция: {position_str}, Вероятность: {probability}')
    else:
        print(f"Для {query} совпадений в документе не найдено")

Документ: 'доченька твоя совсем большая стала'
Запрос: 'дочь'
Результат: Слово - {'text': 'доченька твоя', 'start': 0, 'end': 13}, Позиция: 0 - 13, Вероятность: 0.7695036768913269
Документ: 'вся дорога забита деревьями и цветами'
Запрос: 'дерево'
Результат: Слово - {'text': 'деревьями', 'start': 18, 'end': 27}, Позиция: 18 - 27, Вероятность: 0.6249534249305725
Документ: 'в следующее воскресенье я собираюсь в питер'
Запрос: 'санкт петербург'
Результат: Слово - {'text': 'питер', 'start': 38, 'end': 43}, Позиция: 38 - 43, Вероятность: 0.8686022996902466
Документ: 'у меня сломалась стиралка прикинь'
Запрос: 'стиральная машина'
Результат: Слово - {'text': 'стиралка', 'start': 17, 'end': 25}, Позиция: 17 - 25, Вероятность: 0.8436548948287963
Документ: 'садись в машину и поехали уже'
Запрос: 'автомобиль'
Результат: Слово - {'text': 'машину', 'start': 9, 'end': 15}, Позиция: 9 - 15, Вероятность: 0.8158422231674195
Документ: 'сколько стоит ремонт стиральной машины'
Запрос: 'автомобиль'
Результа